In [47]:
import re, os, json, sys
from os import path as op
import importlib
from collections import defaultdict, OrderedDict

In [10]:
sys.path.append('../../LangPro/python')

In [11]:
import evaluate

In [ ]:
importlib.reload()

In [7]:
os.getcwd()

'/home/kowalsky/Natural Tableau/myNT/nb'

In [13]:
!ls ../results/starSEM2020_ccg_al,ch,w3,-k,r800,c20_ab,ch,cKB,cT,p123.log

../results/starSEM2020_ccg_al,ch,w3,-k,r800,c20_ab,ch,cKB,cT,p123.log


# Functions

In [99]:
def read_langpro_log(fn):
    #fn = '../results/starSEM2020_ccg_al,ch,w3,-k,r800,c20_ab,ch,cKB,cT,p123.log'
    pat = '\s*(\d+):\s*\[\w+\],?\s*(\w+),?\s*'
    with open(fn) as F:
        txt = F.read()
    txt = txt.split('No improvement')[1].split('\n--------')[0]
    preds = re.split('\n\s*(\d+):', txt)[1:]
    preds = list(zip(preds[::2], preds[1::2]))
    predictions = defaultdict(dict)
    for i, p in preds:
        lines = p.split('\n')
        #if i == '2632': print(lines)
        g, a, d = re.match('\s*\[(\w+)\],?\s*(\w+),?\s*(.+)', lines[0]).groups()
        predictions[int(i)] = {'g':g, 's':a, 'i':d}
        if 3 <= len(lines) <= 4:
            predictions[int(i)]['p'] = lines[1].strip()
            predictions[int(i)]['h'] = lines[2].strip()
    return predictions 

In [86]:
read_langpro_log('../results/starSEM2020_ccg_al,ch,w3,-k,r800,c20_ab,ch,cKB,cT,p123.log')

defaultdict(dict,
            {55: {'g': 'yes', 's': 'yes', 'i': "closed, 'Ter',2     XP: []"},
             12: {'g': 'unknown',
              's': 'unknown',
              'i': "open, 'Ter',13    XP: []"},
             42: {'g': 'no', 's': 'no', 'i': "closed, 'Ter',6     XP: []"},
             30: {'g': 'yes', 's': 'yes', 'i': "closed, 'Ter',16    XP: []"},
             49: {'g': 'unknown',
              's': 'unknown',
              'i': "open, 'Ter',52    XP: []"},
             93: {'g': 'unknown',
              's': 'unknown',
              'i': "open, 'Ter',51    XP: []"},
             14: {'g': 'unknown',
              's': 'unknown',
              'i': "open, 'Ter',32    XP: []"},
             18: {'g': 'unknown',
              's': 'unknown',
              'i': "open, 'Ter',38    XP: []"},
             26: {'g': 'unknown',
              's': 'unknown',
              'i': "open, 'Ter',62    XP: []"},
             73: {'g': 'unknown',
              's': 'unknown',
              

# Sandbox

In [100]:
log_files = {sys:f"../results/starSEM2020_{sys}_al,ch,w3,-k,r800,c20_ab,ch,cKB,cT,p123.log"
                 for sys in "ccg eccg depccg".split() }

In [101]:
sys_preds = { sys:read_langpro_log(path) for sys, path in log_files.items() }

In [114]:
failed_probs = dict()
for pid in sys_preds['ccg']:
    #if pid not in [3705]: continue
    predictions = [ preds[pid] for _, preds in sorted(sys_preds.items()) ]
    failed = [ p for p in predictions if p['g'] != p['s'] ]
    #print(failed)
    if len(failed) == len(sys_preds):
    #if len(failed) >= 1:
        f = failed[0]
        d = { 'gold':f['g'], 'anno':"", 'prob':[f['p'], f['h']] }
        d['pred'] = ', '.join([ fp['s'] for fp in predictions ])
        failed_probs[pid] = d
        print(f"{pid}: [{f['g']}]\t{f['s']}\n\t{f['p']}\n\t{f['h']}")
        

90: [no]	unknown
	A man is jumping into an empty pool
	A man is jumping into a full pool
131: [unknown]	yes
	A person is wearing a hat and is sitting on the grass
	A person is sitting in a field and is wearing a hat
256: [no]	unknown
	There is no man on a rock high above some trees standing in a strange position
	A man is on a rock high above some trees and is standing in a strange position
3: [yes]	unknown
	The young boys are playing outdoors and the man is smiling nearby
	The kids are playing outdoors near a man with a smile
100: [yes]	unknown
	Four girls are doing backbends and playing in the garden
	Four girls are doing backbends and playing outdoors
2: [unknown]	no
	A group of children is playing in the house and there is no man standing in the background
	A group of kids is playing in a yard and an old man is standing in the background
77: [yes]	unknown
	People wearing costumes are gathering in a forest and are looking in the same direction
	Masked people are looking in the same 

In [115]:
len(failed_probs)

434

In [107]:
with open('../results/starSEM2020_failed_any.json', 'w') as F:
    json.dump(failed_probs, F, sort_keys=True, indent=1)

In [108]:
sorted(sys_preds)

['ccg', 'depccg', 'eccg']